In [1]:
!pip install pandas
!pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd

In [3]:
general_information = pd.read_excel('data/general_information_bkp.xlsx', engine='openpyxl')